In [1]:
import os

In [2]:
%pwd

'c:\\Users\\91808\\Downloads\\End-to-End-with-mlflow-dvc\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91808\\Downloads\\End-to-End-with-mlflow-dvc'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]=""
os.environ["MLFLOW_TRACKING_USERNAME"]=""
os.environ["MLFLOW_TRACKING_PASSWORD"]=""

In [6]:
import tensorflow as tf


In [7]:
model=tf.keras.models.load_model("artifacts\\training\\model.h5")


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_url:str
    params_image_size:list
    params_batch_size:int

In [9]:
from src.cancerClassification.constants import *
from src.cancerClassification.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts\\training\\model.h5",
            training_data="artifacts\\data_ingestion\\chest-ctscan",
            mlflow_url="https://dagshub.com/Sohail1173/End-to-End-with-mlflow-dvc.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: mlflow in c:\users\91808\downloads\end-to-end-with-mlflow-dvc\myenv\lib\site-packages (2.10.2)



In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-29 08:34:13,107: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 08:34:13,113: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 08:34:13,115: INFO: common: created directory at: artifacts]
Found 104 images belonging to 2 classes.

[2024-02-29 08:34:14,198: WARNING: module_wrapper: From c:\Users\91808\Downloads\End-to-End-with-mlflow-dvc\myenv\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
7/7 [==============================] - 11s 1s/step - loss: 9.5222 - accuracy: 0.4423
[2024-02-29 08:34:25,065: INFO: common: json file saved at: scores.json]


2024/02/29 08:34:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\91808\AppData\Local\Temp\tmpg19_nbmp\model\data\model\assets
[2024-02-29 08:34:32,683: INFO: builder_impl: Assets written to: C:\Users\91808\AppData\Local\Temp\tmpg19_nbmp\model\data\model\assets]


c:\Users\91808\Downloads\End-to-End-with-mlflow-dvc\myenv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/02/29 08:35:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
